# app

rss sources:
https://tw.stock.yahoo.com/rss_index.html

In [1]:
%cd /workspace/twint/app
# !pip install -e .
# !python -m pytest tests/test_scrapers.py::test_cnbc_page_tags -v

/workspace/twint/app


In [1]:
# setup elasticsearch (one-time only)
# %cd /workspace/twint/app
# !python ./app/store/es.py

/workspace/twint/app


In [1]:
%cd /workspace/twint/app
# !chmod +x ./start.sh
# !./start.sh

# !python -m app.main run.scraper=cnbc run.n_workers=1
# !python -m app.main run.scraper=cnyes_page run.n_workers=1 run.max_startpoints=10
# !python -m app.main run.scraper=rss run.n_workers=1 run.loop_every=86400 scraper.rss.entry=./resource/rss_yahoo_us_stock.csv
# !python -m app.main run.scraper=rss run.n_workers=1 run.loop_every=43200 scraper.rss.entry=./resource/rss_yahoo_us_indicies.csv
# !python -m app.main run.scraper=rss run.n_workers=1 run.loop_every=43200 scraper.rss.entry=./resource/rss_yahoo_tw.csv
# !python -m app.main run.scraper=rss run.n_workers=1 run.loop_every=7200 scraper.rss.entry=./resource/rss_news_us.csv
# !python -m app.main run.scraper=moneydj_index run.n_workers=1 scraper.moneydj_index.until=3500 run.startpoints_csv='./outputs/2020-08-09/17-13-53/error_urls.csv'
# !python -m app.main run.scraper=moneydj_index run.n_workers=1
# !python -m app.main run.scraper=moneydj_page run.n_workers=1
!python -m app.main run.scraper=cnbc run.n_workers=1 run.max_startpoints=1000 run.loop_every=3600  run.startpoints_csv=./error_urls.csv


/workspace/twint/app
{'run': {'scraper': 'cnbc', 'n_workers': 1, 'loop_every': 3600, 'sleep_for': 1, 'max_startpoints': 1000, 'startpoints_csv': './error_urls.csv'}, 'proxy': {'enabled': True, 'csv': './resource/proxies.txt'}, 'store': {'es': {'host': 'es:9200'}, 'db': {'user': 'sqlpad', 'password': 'sqlpad', 'host': 'pg', 'dbname': 'test'}}, 'scraper': {'rss': {'entry': './resource/rss_yahoo_us.csv', 'fetch_rss_every_n_seconds': 604800, 'force_fetch': True}, 'cnyes_api': {'start': [2020, 5, 5], 'until': None}, 'moneydj_index': {'start': 1, 'until': 2}}}
[2020-08-17 13:40:27,347][app.scrapers.base][INFO] - scraper start running: 1 workers, loop every 3600 seconds
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/workspace/twint/app/app/main.py", line 52, in <module>
    main()
  File "/usr/local/li

In [1]:
# run single scraper (for testing)
%cd /workspace/twint/app
import nest_asyncio
nest_asyncio.apply()
import asyncio
from hydra.experimental import compose, initialize
from app.scrapers import moneydj
from app.store import es

# initialize(config_dir="./app/app")
cfg = compose("config.yaml")
print(cfg)

es.connect()
scp = moneydj.MoneydjPageScraper(cfg)
asyncio.run(scp.run())

# twint

twitter account: CNBC, CNNBusiness, businessinsider

In [1]:
# %cd /workspace/twint
# !pip install e .
# !twint -u CNBC
# !pip install -U fake-useragent

Requirement already up-to-date: fake-useragent in /usr/local/lib/python3.7/site-packages (0.1.11)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [17]:
import nest_asyncio
nest_asyncio.apply()
import twint

c = twint.Config()
c.Username = "CNBC"
c.Elasticsearch = "http://es:9200"
c.Until='2015-01-01 00:00:00'

# c.Search = "fruit"
twint.run.Search(c)

# elasticsearch

query twint
```json
{
  "_source": [
    "date",
    "username"
  ],
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "username": "business"
          }
        },
        {
          "range": {
            "date": {
              "gt": "2004-01-01 00:00:00",
              "lt": "2023-01-01 00:00:00"
            }
          }
        }
      ]
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [
    {
      "date": "asc"
    }
  ]
}
```

query cnyes
http://localhost:9200/news_page/_search
```json
{
  "query": {
    "bool": {
      "filter": [
        {
          "wildcard": {
            "from_url": "*cnyes.com*"
          }
        },
        {
          "range": {
            "entry_published_at": {
              "gte": "2020-05-01T00:00:00",
              "lt": "2021-01-01T00:00:00"
            }
          }
        }
      ]
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [
    {
      "entry_published_at": "desc"
    }
  ]
}
```

```json
{
  "query": {
    "bool": {
      "filter": [
        {
          "wildcard": {
            "resolved_url": "*cnbc*"
          }
        }
      ]
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [
    {
      "entry_published_at": "desc"
    }
  ]
}
```

In [1]:
%cd /workspace/twint/app/app
import json
from elasticsearch_dsl import connections, Document, Date, Keyword, Q, Search, Text, Range, Integer
from store import es

# query scraper-page
q = Q("wildcard", from_url="www.moneydj.com".lower()) 
# \ & 
#     Q("range", created_at={"gte": "2020-08-05", "lt": None})
# & \ ~Q("term", http_status=200)

es.connect()
s = es.Page.search().filter(q).sort('-created_at')
resp = s.execute()

print(json.dumps(s.to_dict()))
print("-------")
print(resp.hits.total)
for h in resp[0:3]:
    print(h.to_dict())

/workspace/twint/app/app
{"query": {"bool": {"filter": [{"wildcard": {"from_url": "www.moneydj.com"}}]}}, "sort": [{"created_at": {"order": "desc"}}]}
-------
{'value': 80, 'relation': 'eq'}
{'from_url': 'https://www.moneydj.com/KMDJ/News/NewsViewer.aspx?a=e91d494d-51b6-4a77-a8fb-905a5cbd2f82&c=MB010000', 'entry_title': '英國央行：負政策利率目前不是一項有效的經濟刺激工具', 'created_at': '2020-08-08T06:17:50.280029'}
{'from_url': 'https://www.moneydj.com/KMDJ/News/NewsViewer.aspx?a=6a967d07-383f-4da1-a4ca-485a82cddab1&c=MB010000', 'entry_title': '中美晶入股宏捷科成為最大股東 發展氮化鎵', 'created_at': '2020-08-08T06:17:50.263146'}
{'from_url': 'https://www.moneydj.com/KMDJ/News/NewsViewer.aspx?a=5c3cfc6e-f7fa-41c7-9a15-a72237f014f6&c=MB010000', 'entry_title': '《美股》科技股帶動 道瓊上漲0.68% Nasdaq續創新高', 'created_at': '2020-08-08T06:17:50.239180'}


# Elasticsearch Dump

Install nodejs & elasticdump first  
https://github.com/nodesource/distributions/blob/master/README.md  
https://github.com/taskrabbit/elasticsearch-dump  

```bash
curl -sL https://deb.nodesource.com/setup_14.x | sudo -E bash -
sudo apt-get install -y nodejs
npm install elasticdump -g
```

Dump & load  

```bash
# dump
multielasticdump \
  --direction=dump \
  --match='^.*$' \
  --fsCompress \
  --input=http://es:9200 \
  --output=./dump_2020xxxx

# load
multielasticdump \
  --direction=load \
  --match='^.*$' \
  --input=./dump_2020xxxx \
  --output=http://es:9200 \
  --fsCompress

# single

elasticdump \
  --input=http://es:9200/twinttweets \
  --output=./twinttweets_mapping_20200503.json \
  --type=mapping
elasticdump \
  --input=http://es:9200/twinttweets \
  --output=./twinttweets_index_20200503.json \
  --type=data

elasticdump \
  --input=http://es:9200/twinttweets \
  --output=$ \
  | gzip > ./twinttweets_index_20200504.json.gz
  
elasticdump \
  --input=http://es:9200/news_page \
  --output=$ \
  | gzip > ./news_page_index_20200615.json.gz
  
elasticdump \
  --input=./twinttweets_index_20200602.json.gz \
  --output=http://es:9200/twinttweets \
  --fsCompress 
```

# Stock

https://twstock.readthedocs.io/zh_TW/latest/index.html


In [1]:
!pip install twstock

     |████████████████████████████████| 1.9 MB 853 kB/s eta 0:00:01     |█████████████████████▋          | 1.3 MB 853 kB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


# app.tool

In [1]:
%cd /workspace/twint/app

from app import tools

tools.generate_rss_yahoo_csv(
    save_to="./resource/rss_yahoo_us_indicies.csv",
        symbol_path="./resource/symbol_indicies.csv")

/workspace/twint/app
